In [1]:
import pandas as pd
import numpy as np

In [3]:
df0 = pd.read_csv('./after_preprocessing_data/df_final_0627.csv')

In [4]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536097 entries, 0 to 536096
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   자치구                536097 non-null  object 
 1   계약일                536097 non-null  object 
 2   건물면적(㎡)            536097 non-null  float64
 3   층                  536097 non-null  int64  
 4   KOSPI종가            536097 non-null  float64
 5   Fed기준금리            536097 non-null  float64
 6   국제유가(Dubai)($/배럴)  536097 non-null  float64
 7   부동산거래량             536097 non-null  int64  
 8   국고채5년(평균)          536097 non-null  float64
 9   국고채10년(평균)         536097 non-null  float64
 10  회사채3년(평균)          536097 non-null  float64
 11  CD91물(평균)          536097 non-null  float64
 12  BOK기준금리            536097 non-null  float64
 13  건물연식               536097 non-null  int64  
 14  GDP대비가계부채비율        536097 non-null  float64
 15  환율종가               536097 non-null  float64
 16  금값

# Train/Test Split


In [5]:
# Train : 2016-2020
# Test : 2021-2022

df0['계약일'] = pd.to_datetime(df0['계약일'])

In [6]:
train_df = df0.loc[df0['계약일'] < '2021']

In [7]:
test_df = df0.loc[df0['계약일'] >= '2021']

# 전체 DF OLS 확인

In [8]:
# feature & target  split

X = train_df.drop('물건금액(만원)', axis=1)
y = train_df['물건금액(만원)']

In [9]:
# X_train : 숫자형 컬럼만 
X_train = X.select_dtypes([np.number])

---

# Scaling (보류)
1. Features - minMax
2. Target - Log 

In [119]:
def myscaler(df):
    from sklearn.preprocessing import MinMaxScaler, StandardScaler

    df_num = df.select_dtypes([np.number])
    num_col = list(df_num.columns)

    # Scaler 객체 생성
    scaler = StandardScaler()
    x_num_scaled = scaler.fit_transform(df_num)

    # print('\t\t(min, max) (mean, std)')
    print('Scaled (%.2f, %.2f) (%.2f, %.2f)' %(x_num_scaled.min(), x_num_scaled.max(), x_num_scaled.mean(), x_num_scaled.std()))
    x_num_scaled = pd.DataFrame(x_num_scaled, columns=num_col)

    X_scaled = df.copy()
    for i in x_num_scaled:
        X_scaled[i] = x_num_scaled[i]

    return X_scaled

In [120]:
X = train_df.drop('물건금액(만원)', axis=1)
y = train_df['물건금액(만원)']

In [121]:
# X -> minMax
X = myscaler(X)

Scaled (-3.65, 11.47) (-0.00, 1.00)


In [123]:
X_train = X.select_dtypes([np.number])

In [124]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453170 entries, 0 to 453169
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   건물면적(㎡)            453170 non-null  float64
 1   층                  453170 non-null  float64
 2   KOSPI종가            453170 non-null  float64
 3   Fed기준금리            453170 non-null  float64
 4   국제유가(Dubai)($/배럴)  453170 non-null  float64
 5   부동산거래량             453170 non-null  float64
 6   국고채5년(평균)          453170 non-null  float64
 7   국고채10년(평균)         453170 non-null  float64
 8   회사채3년(평균)          453170 non-null  float64
 9   CD91물(평균)          453170 non-null  float64
 10  BOK기준금리            453170 non-null  float64
 11  건물연식               453170 non-null  float64
 12  GDP대비가계부채비율        453170 non-null  float64
 13  환율종가               453170 non-null  float64
 14  금값종가               453170 non-null  float64
 15  선행지수순환변동치          453170 non-null  float64
 16  장단

In [134]:
# y -> log 변환
y = np.log10(y)
y

0         4.255273
1         4.741939
2         4.565848
3         4.829947
4         4.481443
            ...   
453165    5.232996
453166    4.672098
453167    5.243038
453168    4.775974
453169    4.843233
Name: 물건금액(만원), Length: 453170, dtype: float64

In [137]:
# 스케일링 후 predict
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                 2.684e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        13:11:42   Log-Likelihood:             1.2286e+05
No. Observations:              453170   AIC:                        -2.457e+05
Df Residuals:                  453152   BIC:                        -2.455e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.7501      0.000   1.73e+04      0.000       4.750       4.751
건물면적(㎡)               0.1645      0.000    594.683      0.000       0.164       0.165
층                     0.0245      0.000     87.167      0.000       0.024       0.025
KOSPI종가               0.0339      0.001     38.086      0.000       0.032       0.036
Fed기준금리              -0.0069      0.001     -6.052      0.000      -0.009      -0.005
국제유가(Dubai)($/배럴)     0.0218      0.001     24.569      0.000       0.020       0.024
부동산거래량               -0.0133      0.001    -26.036      0.000      -0.014      -0.012
국고채5년(평균)            -0.0041      0.006     -0.727      0.467      -0.015       0.007
국고채10년(평균)           -0.0021      0.004     -0.520      0.603      -0.010       0.006
회사채3년(평균)            -0.0119      0.002     -7.150      0.000      -0.015      -0.009
CD91물(평균)            -0.0211      0.002    -10.283      0.000      -0.025      -0.017
BOK기준금리               0.0101      0.002      5.040      0.000       0.006       0.014
건물연식                 -0.0016      0.000     -5.613      0.000      -0.002      -0.001
GDP대비가계부채비율           0.0498      0.001     42.540      0.000       0.048       0.052
환율종가                  0.0093      0.001     15.298      0.000       0.008       0.011
금값종가                  0.0122      0.001     16.202      0.000       0.011       0.014
선행지수순환변동치            -0.0147      0.001    -10.448      0.000      -0.017      -0.012
장단기금리spread          -0.0053      0.002     -2.470      0.014      -0.010      -0.001
==============================================================================
Omnibus:                    14036.533   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16026.210
Skew:                           0.409   Prob(JB):                         0.00
Kurtosis:                       3.424   Cond. No.                         64.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# OLS_1 : Scaling 후

In [199]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                 2.536e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        13:40:04   Log-Likelihood:             1.2288e+05
No. Observations:              453170   AIC:                        -2.457e+05
Df Residuals:                  453151   BIC:                        -2.455e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.7501      0.000   1.73e+04      0.000       4.750       4.751
건물면적(㎡)               0.1645      0.000    594.634      0.000       0.164       0.165
층                     0.0245      0.000     87.173      0.000       0.024       0.025
KOSPI종가               0.0331      0.001     36.831      0.000       0.031       0.035
Fed기준금리              -0.0072      0.001     -6.316      0.000      -0.009      -0.005
국제유가(Dubai)($/배럴)     0.0226      0.001     25.192      0.000       0.021       0.024
부동산거래량               -0.0139      0.001    -26.724      0.000      -0.015      -0.013
국고채5년(평균)             0.0038      0.006      0.646      0.518      -0.008       0.015
국고채10년(평균)           -0.0050      0.004     -1.208      0.227      -0.013       0.003
회사채3년(평균)            -0.0140      0.002     -8.262      0.000      -0.017      -0.011
CD91물(평균)            -0.0221      0.002    -10.749      0.000      -0.026      -0.018
BOK기준금리               0.0079      0.002      3.899      0.000       0.004       0.012
건물연식                 -0.0016      0.000     -5.703      0.000      -0.002      -0.001
GDP대비가계부채비율           0.0503      0.001     42.868      0.000       0.048       0.053
환율종가                  0.0105      0.001     16.439      0.000       0.009       0.012
금값종가                  0.0115      0.001     15.071      0.000       0.010       0.013
선행지수순환변동치            -0.0153      0.001    -10.868      0.000      -0.018      -0.013
EPU                  -0.0025      0.000     -6.159      0.000      -0.003      -0.002
장단기금리spread          -0.0071      0.002     -3.267      0.001      -0.011      -0.003
==============================================================================
Omnibus:                    14026.821   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16014.587
Skew:                           0.409   Prob(JB):                         0.00
Kurtosis:                       3.423   Cond. No.                         67.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 국고채5년(평균), 국고채10년(평균) column 삭제

In [200]:
X_train.drop(['국고채5년(평균)', '국고채10년(평균)'], axis=1, inplace=True)

# VIF_1

In [201]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_1):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_1.values,i)
                         for i in range(X_train_1.shape[1])]
    vif["Feature"] = X_train_1.columns
    return vif


In [202]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     1.018911            건물면적(㎡)
1     1.051653                  층
2    10.506465            KOSPI종가
3    16.513301            Fed기준금리
4    10.679819  국제유가(Dubai)($/배럴)
5     3.502700             부동산거래량
6    21.039819          회사채3년(평균)
7    32.485511          CD91물(평균)
8    41.100318            BOK기준금리
9     1.048094               건물연식
10   17.719504        GDP대비가계부채비율
11    4.405767               환율종가
12    7.508096               금값종가
13   15.900998          선행지수순환변동치
14    1.988263                EPU
15   27.754332        장단기금리spread


### CD91물 삭제

In [35]:
X_train.drop('CD91물(평균)', axis=1, inplace=True)

# OLS_2

In [36]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                 3.256e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        09:29:36   Log-Likelihood:            -2.5521e+05
No. Observations:              453170   AIC:                         5.105e+05
Df Residuals:                  453155   BIC:                         5.106e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.5406      0.012    782.916      0.000       9.517       9.565
건물면적(㎡)               5.2119      0.009    594.919      0.000       5.195       5.229
층                     0.6627      0.008     87.187      0.000       0.648       0.678
KOSPI종가               0.6048      0.014     44.628      0.000       0.578       0.631
Fed기준금리              -0.0705      0.008     -8.441      0.000      -0.087      -0.054
국제유가(Dubai)($/배럴)     0.2146      0.010     21.980      0.000       0.195       0.234
부동산거래량               -0.1224      0.005    -24.850      0.000      -0.132      -0.113
회사채3년(평균)            -0.1182      0.010    -11.559      0.000      -0.138      -0.098
BOK기준금리              -0.0603      0.012     -5.197      0.000      -0.083      -0.038
건물연식                 -0.0232      0.004     -5.684      0.000      -0.031      -0.015
GDP대비가계부채비율           0.4992      0.010     48.599      0.000       0.479       0.519
환율종가                  0.1144      0.007     17.447      0.000       0.102       0.127
금값종가                  0.1166      0.010     11.540      0.000       0.097       0.136
선행지수순환변동치            -0.1352      0.008    -16.580      0.000      -0.151      -0.119
장단기금리spread          -0.0750      0.014     -5.433      0.000      -0.102      -0.048
==============================================================================
Omnibus:                    14085.728   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16077.888
Skew:                           0.410   Prob(JB):                         0.00
Kurtosis:                       3.422   Cond. No.                         65.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## VIF_2

In [37]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     6.223540            건물면적(㎡)
1     5.068482                  층
2   120.114358            KOSPI종가
3    27.395713            Fed기준금리
4    78.897509  국제유가(Dubai)($/배럴)
5    12.042551             부동산거래량
6    52.473596          회사채3년(평균)
7    60.481235            BOK기준금리
8     5.540367               건물연식
9    44.264996        GDP대비가계부채비율
10   15.084296               환율종가
11   20.763425               금값종가
12   64.905147          선행지수순환변동치
13  133.804770        장단기금리spread


### 장단기금리spread 컬럼 삭제

In [38]:
X_train.drop('장단기금리spread', axis=1, inplace=True)

# OLS_3

In [39]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                 3.506e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        09:29:51   Log-Likelihood:            -2.5523e+05
No. Observations:              453170   AIC:                         5.105e+05
Df Residuals:                  453156   BIC:                         5.106e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.5149      0.011    847.390      0.000       9.493       9.537
건물면적(㎡)               5.2116      0.009    594.880      0.000       5.194       5.229
층                     0.6626      0.008     87.174      0.000       0.648       0.677
KOSPI종가               0.6253      0.013     48.031      0.000       0.600       0.651
Fed기준금리              -0.0699      0.008     -8.368      0.000      -0.086      -0.054
국제유가(Dubai)($/배럴)     0.2010      0.009     21.299      0.000       0.182       0.219
부동산거래량               -0.1280      0.005    -26.550      0.000      -0.137      -0.119
회사채3년(평균)            -0.1638      0.006    -28.101      0.000      -0.175      -0.152
BOK기준금리              -0.0329      0.010     -3.150      0.002      -0.053      -0.012
건물연식                 -0.0229      0.004     -5.598      0.000      -0.031      -0.015
GDP대비가계부채비율           0.5070      0.010     49.842      0.000       0.487       0.527
환율종가                  0.1163      0.007     17.759      0.000       0.103       0.129
금값종가                  0.1171      0.010     11.596      0.000       0.097       0.137
선행지수순환변동치            -0.1634      0.006    -26.018      0.000      -0.176      -0.151
==============================================================================
Omnibus:                    14095.032   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16084.424
Skew:                           0.411   Prob(JB):                         0.00
Kurtosis:                       3.422   Cond. No.                         53.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## VIF_3

In [40]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     6.203726            건물면적(㎡)
1     5.057175                  층
2   116.756452            KOSPI종가
3    23.492250            Fed기준금리
4    66.736951  국제유가(Dubai)($/배럴)
5    11.525542             부동산거래량
6    20.034342          회사채3년(평균)
7    57.286229            BOK기준금리
8     5.532729               건물연식
9    44.152645        GDP대비가계부채비율
10   14.298677               환율종가
11   20.544349               금값종가
12   31.647913          선행지수순환변동치


### KOSPI 종가 컬럼 삭제

In [41]:
X_train.drop('KOSPI종가', axis=1, inplace=True)

# OLS_4

In [42]:
feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 3.760e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        09:30:01   Log-Likelihood:            -2.5638e+05
No. Observations:              453170   AIC:                         5.128e+05
Df Residuals:                  453157   BIC:                         5.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.6581      0.011    889.908      0.000       9.637       9.679
건물면적(㎡)               5.2176      0.009    594.107      0.000       5.200       5.235
층                     0.6662      0.008     87.438      0.000       0.651       0.681
Fed기준금리              -0.0277      0.008     -3.332      0.001      -0.044      -0.011
국제유가(Dubai)($/배럴)     0.1020      0.009     11.045      0.000       0.084       0.120
부동산거래량               -0.0719      0.005    -15.342      0.000      -0.081      -0.063
회사채3년(평균)            -0.0633      0.005    -11.606      0.000      -0.074      -0.053
BOK기준금리               0.0548      0.010      5.314      0.000       0.035       0.075
건물연식                 -0.0216      0.004     -5.259      0.000      -0.030      -0.014
GDP대비가계부채비율           0.7740      0.009     90.636      0.000       0.757       0.791
환율종가                 -0.0034      0.006     -0.561      0.574      -0.015       0.008
금값종가                 -0.0042      0.010     -0.429      0.668      -0.023       0.015
선행지수순환변동치            -0.0203      0.006     -3.654      0.000      -0.031      -0.009
==============================================================================
Omnibus:                    14051.459   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16006.677
Skew:                           0.411   Prob(JB):                         0.00
Kurtosis:                       3.416   Cond. No.                         51.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
# 환율종가 & 금값종가 column 제거
X_train.drop(['환율종가', '금값종가'], axis=1, inplace=True)

## VIF_4

In [44]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor            Feature
0    6.106400            건물면적(㎡)
1    4.973835                  층
2   18.319041            Fed기준금리
3   44.465357  국제유가(Dubai)($/배럴)
4    7.109552             부동산거래량
5    8.829724          회사채3년(평균)
6   22.998294            BOK기준금리
7    5.402440               건물연식
8   11.522906        GDP대비가계부채비율
9    9.652359          선행지수순환변동치


In [45]:
# 국제유가 col 삭제
X_train.drop('국제유가(Dubai)($/배럴)', axis=1, inplace=True)

# OLS_5

In [46]:
feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 5.009e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        09:30:07   Log-Likelihood:            -2.5649e+05
No. Observations:              453170   AIC:                         5.130e+05
Df Residuals:                  453160   BIC:                         5.131e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.6553      0.008   1199.057      0.000       9.639       9.671
건물면적(㎡)         5.2200      0.009    594.367      0.000       5.203       5.237
층               0.6657      0.008     87.356      0.000       0.651       0.681
Fed기준금리         0.0187      0.006      3.046      0.002       0.007       0.031
부동산거래량         -0.0572      0.004    -14.600      0.000      -0.065      -0.050
회사채3년(평균)      -0.0495      0.004    -13.921      0.000      -0.057      -0.043
BOK기준금리         0.0753      0.010      7.642      0.000       0.056       0.095
건물연식           -0.0225      0.004     -5.490      0.000      -0.031      -0.014
GDP대비가계부채비율     0.7897      0.008     95.550      0.000       0.774       0.806
선행지수순환변동치      -0.0067      0.003     -1.941      0.052      -0.013    6.66e-05
==============================================================================
Omnibus:                    14069.875   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16030.565
Skew:                           0.411   Prob(JB):                         0.00
Kurtosis:                       3.417   Cond. No.                         39.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## VIF_5

In [47]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor      Feature
0    6.103294      건물면적(㎡)
1    4.973835            층
2   11.289780      Fed기준금리
3    6.522285       부동산거래량
4    8.119077    회사채3년(평균)
5   20.692597      BOK기준금리
6    5.401891         건물연식
7   10.833742  GDP대비가계부채비율
8    9.129485    선행지수순환변동치


In [48]:
# BOK기준금리 제거
X_train.drop('BOK기준금리', axis=1, inplace=True)

# OLS_6

In [49]:
feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 5.633e+04
Date:                Tue, 27 Jun 2023   Prob (F-statistic):               0.00
Time:                        09:30:11   Log-Likelihood:            -2.5652e+05
No. Observations:              453170   AIC:                         5.131e+05
Df Residuals:                  453161   BIC:                         5.132e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.7105      0.004   2733.612      0.000       9.704       9.717
건물면적(㎡)         5.2192      0.009    594.281      0.000       5.202       5.236
층               0.6655      0.008     87.315      0.000       0.651       0.680
Fed기준금리         0.0613      0.003     24.294      0.000       0.056       0.066
부동산거래량         -0.0651      0.004    -17.199      0.000      -0.072      -0.058
회사채3년(평균)      -0.0451      0.004    -12.852      0.000      -0.052      -0.038
건물연식           -0.0225      0.004     -5.495      0.000      -0.031      -0.014
GDP대비가계부채비율     0.7336      0.004    193.630      0.000       0.726       0.741
선행지수순환변동치      -0.0150      0.003     -4.556      0.000      -0.021      -0.009
==============================================================================
Omnibus:                    14074.026   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16025.652
Skew:                           0.411   Prob(JB):                         0.00
Kurtosis:                       3.415   Cond. No.                         20.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
df0['층'].describe()

count    536097.000000
mean          9.088667
std           6.146602
min          -3.000000
25%           4.000000
50%           8.000000
75%          13.000000
max          69.000000
Name: 층, dtype: float64

In [51]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor      Feature
0    5.362790      건물면적(㎡)
1    4.384840            층
2    3.560076      Fed기준금리
3    6.299096       부동산거래량
4    8.092153    회사채3년(평균)
5    4.469694         건물연식
6    7.333581  GDP대비가계부채비율
7    8.460557    선행지수순환변동치


In [52]:
import numpy as np 
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size = 0.2)

from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test,y_pred)

print('MSE : {0:}, RMSE : {1: .3f}, MAE:{1: .3f}'.format(mse, rmse, mae))
print('Variance score : {0: .3f}'.format(r2_score(y_test, y_pred)))

MSE : 0.18156684472670578, RMSE :  0.426, MAE: 0.426
Variance score :  0.498


In [53]:
np.log(100)

4.605170185988092

In [54]:
np.log(df0['물건금액(만원)'])

0          9.798127
1         10.918718
2         10.513253
3         11.121363
4         10.318903
            ...    
536092    12.429216
536093    11.835009
536094    12.055250
536095    10.518673
536096    11.245046
Name: 물건금액(만원), Length: 536097, dtype: float64

-----

## 스케일링OLS -> VIF 반복 후 -> 최종 OLS 확인

# X_test column 제거 
- X_train에서 날린 컬럼들 모두 제거

# DF split 
## 강남/ 비강남 DF 

# Modeling 


## 1. OLS 확인 
- P-value확인 후 컬럼 지우기 
- coef: feature별 계수 
